# PyTorch - ciąg dalszy

## Przykład: trenowanie prostego modelu
Przykład jest zaczerpnięty z [oficjalnego tutoriala](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html).

In [1]:
import time

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [22]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

100%|██████████| 26421880/26421880 [00:09<00:00, 2860768.35it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 84497.76it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:02<00:00, 1850960.47it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 6497826.36it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


Dygresja: DataLoadery.

Gdy używamy CPU i GPU, to nie jest trywialne jak zaimplementować ładowanie danych i podawanie ich do modelu, by zarówno wykorzystanie CPU jak i GPU było optymalne. Przy naiwnej implementacji możemy mieć następujący efekt:
![](https://i.imgur.com/b3Oxcf1.png)

Wolelibyśmy, żeby ten obrazek wyglądał tak:
![](https://i.imgur.com/QRGSW6Z.png)

Na szczęście PyTorch robi całą tę brudną robotę za nas, i udostępnia klasy `Dataset` i `DataLoader`, które umożliwiają efektywne wielowątkowe ładowanie danych, batchowanie itp.

In [87]:
## I moved my dropout here, so I can test if it works in real, sequential torch model

class DropoutFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, p):
        p = p.to(input.device)
        D = (torch.rand(input.shape).to(input.device) > p).float()
        ctx.save_for_backward(input, D, p)
        return (input * D) / (1 - p)

    @staticmethod
    def backward(ctx, grad_output):
        input, D, p = ctx.saved_tensors
        A = grad_output * D
        return A / (1-p), torch.tensor([0])

class MyDropout(nn.Module):
    def __init__(self, p=0.25):
        super(MyDropout, self).__init__()
        self.p = torch.tensor([p])

    def forward(self,x):
        if self.training:
            return DropoutFunction.apply(x, self.p)
        else:
            return x

In [88]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.BatchNorm1d(512),  # Batch Normalization after the first Linear layer
            nn.ReLU(),
            MyDropout(0.2),
            nn.Linear(512, 356),
            nn.BatchNorm1d(356),  # Batch Normalization after the second Linear layer
            nn.ReLU(),
            MyDropout(0.3),
            nn.Linear(356, 128),
            nn.BatchNorm1d(128),  # Batch Normalization after the third Linear layer
            nn.ReLU(),
            MyDropout(0.2),
            nn.Linear(128, 10),
        )


    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MyDropout()
    (4): Linear(in_features=512, out_features=356, bias=True)
    (5): BatchNorm1d(356, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MyDropout()
    (8): Linear(in_features=356, out_features=128, bias=True)
    (9): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MyDropout()
    (12): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [89]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [90]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()  # ważny krok! nie chcemy żeby gradienty z różnych kroków się na siebie "nakładały"
        loss.backward()
        optimizer.step()

        if batch % 400 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 30
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    start_time = time.time()
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
    elapsed_time = time.time() - start_time
    print(f"Epoch finished in {elapsed_time:.1f}s.\n\n")
print("Done!")

Epoch 1
-------------------------------
loss: 2.402184  [    0/60000]
loss: 0.678566  [25600/60000]
loss: 0.553627  [51200/60000]
Test Error: 
 Accuracy: 84.8%, Avg loss: 0.006879 

Epoch finished in 10.9s.


Epoch 2
-------------------------------
loss: 0.380755  [    0/60000]
loss: 0.440382  [25600/60000]
loss: 0.445928  [51200/60000]
Test Error: 
 Accuracy: 85.7%, Avg loss: 0.006277 

Epoch finished in 11.1s.


Epoch 3
-------------------------------
loss: 0.351271  [    0/60000]
loss: 0.348417  [25600/60000]
loss: 0.416932  [51200/60000]
Test Error: 
 Accuracy: 87.0%, Avg loss: 0.005561 

Epoch finished in 12.9s.


Epoch 4
-------------------------------
loss: 0.238435  [    0/60000]
loss: 0.310225  [25600/60000]
loss: 0.333073  [51200/60000]
Test Error: 
 Accuracy: 87.7%, Avg loss: 0.005299 

Epoch finished in 15.7s.


Epoch 5
-------------------------------
loss: 0.231587  [    0/60000]
loss: 0.419916  [25600/60000]
loss: 0.366704  [51200/60000]
Test Error: 
 Accuracy: 87.7%, Avg

## Przykład: różniczkowanie mnożenia
Przypomnijmy sobie regułę łańcuchową liczenia pochodnych. Jeśli mamy:
$$ L(x) = g(f(x)), $$
to wtedy:

$$ \frac{\partial L(x)}{dx} = \frac{\partial L(x)}{\partial f(x)}\frac{\partial f(x)}{\partial x} $$

W kontekście automatycznego różniczkowania w PyTorchu kluczowa jest tu właściwość, że do policzenia gradientu nie musimy nic wiedzieć o funkcji $g$ o ile tylko znamy $\frac{\partial L(x)}{\partial f(x)}$. **Każdy moduł wie, jak policzyć swój gradient i dzięki temu można łańcuchowo liczyć pochodne skomplikowanych funkcji.**


W PyTorchu każda funkcja, której używamy, ma zaimplementowane dwa podmoduły:
* **Forward** - na podstawie podanego $x$ potrafi obliczyć $f(x)$.
* **Backward** - na podstawie podanego $\frac{\partial L(x)}{\partial f(x)}$ potrafi policzyć $\frac{\partial L(x)}{\partial x}$.

Obiekt, który reprezentuje w pytorchu funkcję, która potrafi policzyć swoją pochodną, dziedziczy po klasie `torch.autograd.Function`.

Chcemy zaimplementować od nowa w PyTorchu fukcję $f(a, b) = a \cdot b$, która potrafi policzyć swoje pochodne.

W efekcie implementujemy obiekt typu `torch.autograd.Function` z metodami:
* **Forward**
    1. Dostaje na wejściu `a` oraz `b`
    1. Zapamiętuje `a` oraz `b`, które przydadzą się później przy liczeniu pochodnej
    2. Zwraca `a * b`
* **Backward**
    1. Dostaje na wejściu `grad_output` reprezentujące wartość $\frac{\partial L(x)}{\partial f(a, b)}$.
    2. Wyjmuje z pamięci `a` oraz `b`.
    3. Liczy swoją pochodną po a: $\frac{\partial f(a, b)}{\partial a} = \frac{ \partial ab }{\partial a} = b$
    4. Liczy swoją pochodną po b: $\frac{\partial f(a, b)}{\partial b} = \frac{ \partial ab }{\partial b} = a$
    5. Zwraca pochodne $\frac{\partial L(x)}{\partial f(a, b)} \frac{\partial f(a, b)}{\partial a}$ oraz $\frac{\partial L(x)}{\partial f(a, b)} \frac{\partial f(a, b)}{db}$.

In [ ]:
class MyProduct(torch.autograd.Function):
    @staticmethod
    def forward(self, a, b):
        self.save_for_backward(a, b)
        return a * b

    @staticmethod
    def backward(self, grad_output):
        # Wyjmujemy z pamięci a oraz b
        a, b = self.saved_tensors
        # Liczymy pochodną po a
        a_grad = b
        # Liczymy pochodną po b
        b_grad = a

        # Zwracamy "łańcuchowe" pochodne
        return grad_output * a_grad, grad_output * b_grad

In [ ]:
prod = MyProduct.apply  # bierzemy faktyczną funkcję, którą możemy odpalać na tensorach

x = torch.tensor(2., requires_grad=True)
y = torch.tensor(3., requires_grad=True)
z = torch.tensor(5., requires_grad=True)

result = prod(prod(x, y), z)  # sekwencyjne użycie - pytorch sam ogarnie składanie operatorów

result.backward()  # uruchamiamy backpropagation
print(x.grad)

## Zadanie

**Zadanie 1.** (*20% punktów*) Proszę zaimplementować funkcje `MySum`, `MySigmoid`, `MyRelu`, dziedziczące po `torch.autograd.Function` i implementujące odpowiednio operacje: sumy dwóch tensorów, operacji $\sigma$ (sigmoidy), aktywacji relu.

In [ ]:
class MySum(torch.autograd.Function):
    @staticmethod
    def forward(self,a,b):
        self.save_for_backward(a,b)
        return a + b

    @staticmethod
    def backward(self, grad_output):
        a,b = self.saved_tensors

        grad_a = 1
        grad_b = 1

        return grad_output * grad_a, grad_output * grad_b

In [ ]:
import torch

class MySigmoid(torch.autograd.Function):
    @staticmethod
    def forward(ctx, a):
        sigmoid_value = 1 / (1 + torch.exp(-a))

        ctx.save_for_backward(a)

        return sigmoid_value

    @staticmethod
    def backward(ctx, grad_output):
        a, = ctx.saved_tensors

        sigmoid_value = 1 / (1 + torch.exp(-a))

        grad_a = sigmoid_value * (1 - sigmoid_value)

        return grad_output * grad_a

In [ ]:
class MyRelu(torch.autograd.Function):
    @staticmethod
    def forward(self,a):
        self.save_for_backward(a)
        return torch.maximum(a,torch.tensor(0.0))

    @staticmethod
    def backward(self, grad_output):
        a = self.saved_tensors

        grad_a = (x > 0).float()

        return grad_output * grad_a

In [ ]:
prod = MySigmoid.apply  # bierzemy faktyczną funkcję, którą możemy odpalać na tensorach

x = torch.tensor(10., requires_grad=True)

result = prod(x)

result.backward()
print(x.grad)

## Przykład: moduły

Moduły (`nn.Module`) to coś więcej niż funkcje (`torch.autograd.Function`). Moduły zarządzają też parametrami i są podstawowymi cegiełkami, z których budujemy sieć neuronową. Np. możemy mieć moduł "warstwa liniowa", "warstwa konwolucyjna" itp.

Typowo, moduł w środku woła funkcje (`torch.autograd.Function`), by potem mogła zostać wykonana propagacja wsteczna.

Prześledźmy poniżej przykład z [dokumentacji PyTorcha](https://pytorch.org/docs/stable/notes/extending.html#extending-torch-autograd), gdzie jest zaimplementowana warstwa liniowa. *Uwaga*: jest to tylko przykład dla celów edukacyjnych, a nie faktyczna implementacja użyta w bibliotece PyTorch.

Ponieważ logika liczenia pochodnych jest już zaszyta w funkcjach (`torch.autograd.Function`), to w module wystarczy zaimplementować tylko funkcję `forward`!

In [2]:
# Inherit from Function
class LinearFunction(torch.autograd.Function):

    # Note that both forward and backward are @staticmethods
    @staticmethod
    # bias is an optional argument
    def forward(ctx, input, weight, bias=None):
        ctx.save_for_backward(input, weight, bias)
        output = input.mm(weight.t())
        if bias is not None:
            output += bias.unsqueeze(0).expand_as(output)
        return output

    # This function has only a single output, so it gets only one gradient
    @staticmethod
    def backward(ctx, grad_output):
        # This is a pattern that is very convenient - at the top of backward
        # unpack saved_tensors and initialize all gradients w.r.t. inputs to
        # None. Thanks to the fact that additional trailing Nones are
        # ignored, the return statement is simple even when the function has
        # optional inputs.
        input, weight, bias = ctx.saved_tensors
        grad_input = grad_weight = grad_bias = None

        # These needs_input_grad checks are optional and there only to
        # improve efficiency. If you want to make your code simpler, you can
        # skip them. Returning gradients for inputs that don't require it is
        # not an error.
        if ctx.needs_input_grad[0]:
            grad_input = grad_output.mm(weight)
        if ctx.needs_input_grad[1]:
            grad_weight = grad_output.t().mm(input)
        if bias is not None and ctx.needs_input_grad[2]:
            grad_bias = grad_output.sum(0)

        return grad_input, grad_weight, grad_bias

linear = LinearFunction.apply

In [9]:
class Linear(nn.Module):
    def __init__(self, input_features, output_features, bias=True):
        super(Linear, self).__init__()
        self.input_features = input_features
        self.output_features = output_features

        # nn.Parameter is a special kind of Tensor, that will get
        # automatically registered as Module's parameter once it's assigned
        # as an attribute. Parameters and buffers need to be registered, or
        # they won't appear in .parameters() (doesn't apply to buffers), and
        # won't be converted when e.g. .cuda() is called. You can use
        # .register_buffer() to register buffers.
        # nn.Parameters require gradients by default.
        self.weight = nn.Parameter(torch.Tensor(output_features, input_features))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(output_features))
        else:
            # You should always register all possible parameters, but the
            # optional ones can be None if you want.
            self.register_parameter('bias', None)

        # Not a very smart way to initialize weights
        self.weight.data.uniform_(-0.1, 0.1)
        if self.bias is not None:
            self.bias.data.uniform_(-0.1, 0.1)

    def forward(self, input):
        # See the autograd section for explanation of what happens here.
        return LinearFunction.apply(input, self.weight, self.bias)

    def extra_repr(self):
        # (Optional)Set the extra information about this module. You can test
        # it by printing an object of this class.
        return 'input_features={}, output_features={}, bias={}'.format(
            self.input_features, self.output_features, self.bias is not None
        )

In [12]:
linear_layer = Linear(10, 20)
print(linear_layer)

linear_layer(torch.tensor([[1.,2.,3.,4.,5.,6.,7.,8.,9., 9.]]))

Linear(input_features=10, output_features=20, bias=True)


tensor([[-0.1029,  0.8311,  0.0362, -1.2362, -0.9078, -0.0261,  0.0188,  1.2537,
          1.2374,  0.9751,  0.3222,  0.4445,  0.0967, -2.2882,  0.1282, -1.2846,
          0.1086,  1.4780,  0.5609,  0.3851]],
       grad_fn=<LinearFunctionBackward>)

## Zadania

**Zadanie 2.** (*40% punktów*) Proszę zaimplementować pytorchowe funkcję oraz moduł do liczenia dropoutu. Wartość $p$ dropoutu powinna być ustawialna jako parametr modułu. Moduł powinien w trybie ewaluacji nic nie robić, a w trybie treningu stosować dropout i skalować aktywacje razy $\frac{1}{1 - p}$.

W celu przetestowania modułu, proszę go dorzucić (bezpośrednio po pierwszej i drugiej warstwie liniowej) do przykładowej architektury przedstawionej wyżej (sekcja "trenowanie prostego modelu"):
* gdy ustawimy `p=0`, nic nie powinno się zmienić;
* gdy ustawimy `p=0.2`, ostateczny wynik na zbiorze testowym powinien być wyższy;
* gdy ustawimy `p=1`, trenowanie powinno się popsuć.

*Uwaga*. Nie wolno korzystać z gotowych pytorchowych funkcji typu `torch.nn.functional.dropout`.

*Podpowiedź*. By zróżnicować zachowanie modułu w trakcie treningu i ewaluacji, [możemy wykorzystać](https://discuss.pytorch.org/t/how-do-i-customize-my-modules-behavior-for-train-and-eval/52693) atrybut `self.training`.

In [21]:
class DropoutFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, p):
        D = (torch.rand(input.shape) > p).float()
        ctx.save_for_backward(input, D, p)
        return (input * D) / (1 - p)

    @staticmethod
    def backward(ctx, grad_output):
        input, D, p = ctx.saved_tensors
        A = grad_output * D
        return A / (1-p)

class MyDropout(nn.Module):
    def __init__(self, p=0.25):
        super(MyDropout, self).__init__()
        self.p = torch.tensor([p])

    def forward(self,x):
        return DropoutFunction.apply(x, self.p)

input = torch.tensor([[2,3,4,5,3,2,23,3,34,3,23,2,3233,3,3435,3,234]])
mD = MyDropout(p=0.4)
output = mD(input)

print(output)


tensor([[3.3333e+00, 5.0000e+00, 6.6667e+00, 8.3333e+00, 5.0000e+00, 0.0000e+00,
         3.8333e+01, 5.0000e+00, 0.0000e+00, 5.0000e+00, 3.8333e+01, 3.3333e+00,
         0.0000e+00, 0.0000e+00, 5.7250e+03, 0.0000e+00, 0.0000e+00]])


**Zadanie 3.** (*40% punktów*) Proszę zmodyfikować przykład z sekcji "trenowanie prostego modelu" tak, by osiągnąć accuracy na zbiorze testowym równe przynajmniej **88.5%**. Można używać gotowych modułów zaimplementowanych w pytorchu. Proszę pozostać przy warstwach liniowych, zabronione jest używanie warstw konwolucyjnych.

*Podpowiedź*. Warto pobawić się następującymi elementami: dropout, batch norm, optymalizator (jakiś inny zamiast SGD).

*Podpowiedź 2*. Proszę się przełączyć na GPU (Środowisko wykonawcze->Zmień typ środowiska wykonawczego->Akcelerator sprzętowy->GPU). Na tym modelu daje to około 2.5x przyspieszenie. (Przyspieszenie jest większe dla większych modeli!)

*Podpowiedź 3*. W oryginalnej architekturze jest pewien niepotrzebny element, którego usunięcie od razu powoduje poprawienie modelu.